In [ ]:
#!pip install accelerate -U

In [4]:
!pip install langgraph langchain langchain-google-genai duckduckgo-search
!pip install langchain-community langchain-core
!pip install -U ddgs

INFO: pip is looking at multiple versions of langchain-google-genai to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 89.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.6/207.6 kB 12.0 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not current

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.2/467.2 kB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.79
    Uninstalling langchain-core-0.3.79:
      Successfully uninstalled langchain-core-0.3.79
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.11
    Uninstalling langchain-text-splitters-0.3.11:
      Successfully uninstalled langchain-text-splitters-0.3.11
ERROR: pip's dependency resolver

In [1]:
import os
from google.colab import userdata

# Nạp API key từ Colab Secrets
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
from typing import TypedDict, List

class AgentState(TypedDict):
    topic: str              
    research: str           
    draft: str              
    critique: str          

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.tools import DuckDuckGoSearchRun

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash",
                           temperature=0,
                           )


search_tool = DuckDuckGoSearchRun()


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser


# ----- NODE 1: AGENT RESEARCHER -----
def researcher_node(state: AgentState):
    """
    Agent này nhận chủ đề (topic) từ state,
    dùng search_tool để tìm kiếm, và trả về kết quả nghiên cứu (research).
    """
    print("--- 🔍 Node: Researcher (Đang nghiên cứu) ---")
    topic = state['topic']

    # Sử dụng công cụ tìm kiếm
    research_result = search_tool.run(topic)

    # Trả về một dictionary để cập nhật state
    return {"research": research_result}


def writer_node(state: AgentState):
    """
    Agent nay nhan thong tin va VIET LAI neu bi phe binh.
    """
    print("--- Node: Writer (Dang viet bai) ---")
    topic = state['topic']
    research = state['research']
    critique = state.get('critique')

    if not critique:
        # Neu chua co critique (lan viet dau tien)
        prompt_template = ChatPromptTemplate.from_template(
            "Ban la mot blogger chuyen nghiep. Dua tren thong tin nghien cuu sau, "
            "hay viet mot bai blog ngan (khoang 150-200 tu) ve chu de: {topic}.\n\n"
            "QUAN TRONG: Chi viet van ban thuan tuy (plain text). "
            "KHONG su dung bat ky dinh dang Markdown nao (vi du: khong dung ** , ## , * , - ).\n\n"
            "Thong tin nghien cuu:\n{research}"
        )
    else:
        # Neu da co critique (phai viet lai)
        print("--- Node: Writer (Bi che! Dang viet lai...) ---")
        prompt_template = ChatPromptTemplate.from_template(
            "Ban la mot blogger chuyen nghiep. Ban nhap truoc cua ban da bi phe binh. "
            "Dua tren nhan xet sau, hay viet lai bai blog cho tot hon.\n\n"
            "QUAN TRONG: Chi viet van ban thuan tuy (plain text). "
            "KHONG su dung bat ky dinh dang Markdown nao (vi du: khong dung ** , ## , * , - ).\n\n"
            "Chu de: {topic}\n"
            "Thong tin nghien cuu ban dau:\n{research}\n\n"
            "Nhan xet (Critique) can sua:\n{critique}"
        )

    # Tao chuoi chain
    writer_chain = prompt_template | llm | StrOutputParser()

    draft = writer_chain.invoke({
        "topic": topic,
        "research": research,
        "critique": critique
    })

    return {"draft": draft}

# ----- NODE 3: AGENT CRITIC -----
def critic_node(state: AgentState):
    """
    Agent này nhận bản nháp (draft) từ state và đưa ra nhận xét (critique).
    Quan trọng: Nó phải quyết định 'ĐẠT' hoặc 'CHƯA ĐẠT'.
    """
    print("--- Node: Critic (Đang phê bình) ---")
    draft = state['draft']

    # Prompt để hướng dẫn LLM phê bình
    prompt_template = ChatPromptTemplate.from_template(
        "Bạn là một nhà phê bình blog khó tính. Hãy đọc bản nháp sau và đưa ra nhận xét.\n"
        "Nếu bản nháp đã tốt, logic, và đủ thông tin, hãy nói 'ĐẠT'.\n"
        "Nếu bản nháp cần cải thiện (ví dụ: thiếu thông tin, viết tệ, lạc đề), "
        "hãy chỉ ra các điểm yếu và nói 'CHƯA ĐẠT'.\n\n"
        "KHONG su dung bat ky dinh dang Markdown nao (vi du: khong dung ** , ## , * , - ).\n\n"
        "Bản nháp:\n{draft}"
    )

    critic_chain = prompt_template | llm | StrOutputParser()

    critique = critic_chain.invoke({"draft": draft})

    print(f"Nhận xét: {critique}")

    return {"critique": critique}

In [11]:
from langgraph.graph import StateGraph, END

def should_continue(state: AgentState):
    """
    Hàm quyết định luồng đi tiếp theo sau khi Critic làm việc.
    """
    print("--- Node: Logic (Quyết định) ---")
    critique = state['critique']

    if "CHƯA ĐẠT" in critique:
        # Nếu bị chê -> Quay lại bước "writer"
        print("--- 👎 Phê bình: CHƯA ĐẠT. Yêu cầu viết lại! ---")
        return "writer"
    else:
        # Nếu được khen -> Kết thúc
        print("--- 👍 Phê bình: ĐẠT. Hoàn thành! ---")
        return END

In [ ]:
# 1. Khởi tạo một "Hội đồng" trống, nói cho nó biết "bàn làm việc" là AgentState
workflow = StateGraph(AgentState)

# 2. Thêm các agent (nodes) vào "Hội đồng"
workflow.add_node("researcher", researcher_node) 
workflow.add_node("writer", writer_node)      
workflow.add_node("critic", critic_node)     

# 3. Đặt điểm BẮT ĐẦU
workflow.set_entry_point("researcher")

# 4. Vẽ các đường nối (edges)
# Sau khi "researcher" làm xong -> chuyển cho "writer"
workflow.add_edge("researcher", "writer")

# Sau khi "writer" làm xong -> chuyển cho "critic"
workflow.add_edge("writer", "critic")

# 5. Thêm đường nối CÓ ĐIỀU KIỆN (Logic rẽ nhánh)
workflow.add_conditional_edges(
    # Bắt đầu từ node "critic"
    "critic",
    # Gọi hàm "should_continue" để quyết định đi đâu
    should_continue,
    # Ánh xạ các kết quả trả về của hàm:
    {
        "writer": "writer", 
        END: END           
    }
)
app = workflow.compile()

In [13]:
# 1. Yeu cau nguoi dung nhap chu de
user_topic = input("Vui long nhap chu de ban muon 'Hoi dong' thuc hien: ")

# 2. Tao dictionary 'inputs' tu chu de do
inputs = {"topic": user_topic}

print(f"\n--- Dang giao viec cho 'Hoi dong' voi chu de: \"{inputs['topic']}\" ---")
print("Vui long cho...")

# 3. Chay .invoke()
final_state = app.invoke(inputs)

print("\n" + "="*50)
print("--- HOI DONG DA HOAN THANH CONG VIEC ---")
print("="*50 + "\n")

# 4. In ket qua cuoi cung
print("BAI VIET HOAN CHINH (PHIEN BAN DA DUOC DUYET):\n")
print(final_state['draft'])

Vui long nhap chu de ban muon 'Hoi dong' thuc hien: Làm thế nào để bắt đầu học chơi guitar cho người mới?

--- Dang giao viec cho 'Hoi dong' voi chu de: "Làm thế nào để bắt đầu học chơi guitar cho người mới?" ---
Vui long cho...
--- 🔍 Node: Researcher (Đang nghiên cứu) ---
--- Node: Writer (Dang viet bai) ---
--- Node: Critic (Đang phê bình) ---
Nhận xét: CHƯA ĐẠT

Bản nháp này có một tiêu đề rất hấp dẫn và một giọng văn thân thiện, dễ chịu, tạo cảm giác đồng cảm với người đọc. Logic trình bày cũng khá mạch lạc: nhận diện vấn đề (choáng ngợp, sai lầm phổ biến), chỉ ra hậu quả (nản chí), và đưa ra lời khuyên chung (tìm lộ trình, chọn đàn).

Tuy nhiên, với vai trò là một nhà phê bình khó tính, tôi thấy bản nháp này còn thiếu rất nhiều thông tin quan trọng và mang tính hành động mà một người mới bắt đầu thực sự cần. Tiêu đề là "Bắt Đầu Học Guitar Cho Người Mới", nhưng nội dung lại thiên về "cách tiếp cận việc học guitar" hơn là "những bước bắt đầu cụ thể".

Các điểm yếu cần cải thiện:

1.